In [1]:
import numpy as np
import pandas as pd

from statistics import mean

from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import (
    cross_validate, train_test_split, GridSearchCV, learning_curve, validation_curve
)
from sklearn.metrics import classification_report, accuracy_score

# Import dataset

In [2]:
compressed_final = pd.read_csv("../DataFormating/compressed_final.csv")
final = pd.read_csv("../DataFormating/final.csv")

# Setup `X`, `y` data for training / testing

In [3]:
compressed_final.columns[:10]

Index(['Stage', 'Home Team Name', 'Home Team Goals', 'Away Team Goals',
       'Away Team Name', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

In [4]:
X = compressed_final.drop(["Home Team Goals", "Away Team Goals", 
                           "Half-time Home Goals", "Half-time Away Goals", 
                           "Home Team Initials", "Away Team Initials"], axis=1)
y = []

In [5]:
for i in range(len(compressed_final)):
    home_team_goals = compressed_final.iloc[i]["Home Team Goals"]
    away_team_goals = compressed_final.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
X["Stage"] = LabelEncoder().fit_transform(X["Stage"])
X["Home Team Name"] = LabelEncoder().fit_transform(X["Home Team Name"])
X["Away Team Name"] = LabelEncoder().fit_transform(X["Away Team Name"])

In [8]:
len(X.columns)

413

### Feature Selection

In [9]:
X.columns[4:]

Index(['Player 1 Age Diff', 'Player 1 Overall Diff', 'Player 1 Potential Diff',
       'Player 1 Acceleration Diff', 'Player 1 Aggression Diff',
       'Player 1 Agility Diff', 'Player 1 Balance Diff',
       'Player 1 Ball control Diff', 'Player 1 Composure Diff',
       'Player 1 Crossing Diff',
       ...
       'Player 11 Shot power Diff', 'Player 11 Sliding tackle Diff',
       'Player 11 Sprint speed Diff', 'Player 11 Stamina Diff',
       'Player 11 Standing tackle Diff', 'Player 11 Strength Diff',
       'Player 11 Vision Diff', 'Player 11 Volleys Diff',
       'Mean Home Team Goals', 'Mean Away Team Goals'],
      dtype='object', length=409)

In [10]:
# selection = SelectKBest(score_func=f_classif, k=20)
# selection.fit(X, y)

# X = selection.transform(X)

In [11]:
feature_names = [
    "Stage", "Home Team Name", "Away Team Name", "Attendance",
    "Overall", # "Potential",
    "Mean Home Team Goals", "Mean Away Team Goals"
]

COLUMNS = []

for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]

In [12]:
X

,Stage,Home Team Name,Away Team Name,Attendance,Player 1 Overall Diff,Player 2 Overall Diff,Player 3 Overall Diff,Player 4 Overall Diff,Player 5 Overall Diff,Player 6 Overall Diff,Player 7 Overall Diff,Player 8 Overall Diff,Player 9 Overall Diff,Player 10 Overall Diff,Player 11 Overall Diff,Mean Home Team Goals,Mean Away Team Goals
0,1,34,24,84490.0,-8,-4,7,-18,16,16,7,12,12,-12,-20,1.000000,1.000000
1,1,38,14,64100.0,-1,24,23,10,-5,-12,3,5,-6,3,14,0.500000,0.000000
2,2,23,17,31513.0,-4,-2,-3,-17,-9,-21,0,-19,4,-6,-21,2.000000,0.000000
3,2,1,27,55686.0,-6,-6,-18,12,-1,21,0,-14,10,-15,-8,2.000000,1.666667
4,3,13,37,38646.0,15,0,-14,7,11,-1,-11,-9,-15,14,-1,1.000000,1.000000
5,3,0,33,30325.0,8,-4,25,-23,-8,25,18,3,-12,-10,-14,1.000000,0.500000
6,4,31,16,38833.0,-3,-2,-9,-3,-2,6,-9,-9,-2,6,-9,0.000000,1.000000
7,4,15,2,62660.0,-3,-14,6,-15,-10,-11,-3,-11,-6,-5,-6,2.333333,1.333333
8,5,25,11,83465.0,12,6,-5,8,10,2,-13,22,-13,18,6,1.000000,0.500000
9,5,21,5,30620.0,20,-4,-5,-14,0,-9,20,-11,2,-23,21,1.000000,0.000000


### Split `X` and `y` into train / test sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Fast testing

In [13]:
def test_model(model, cv=10):
    cv_scores = cross_validate(model, X, y, cv=cv)
    
    mean_train_acc = mean(cv_scores["train_score"]) 
    mean_test_acc = mean(cv_scores["test_score"])
    
    print()
    print("Train Accuracy: ", mean_train_acc)
    print("Test Accuracy: ", mean_test_acc)
    print()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

### K-Nearest Neighbors

In [14]:
test_model(KNeighborsClassifier(n_neighbors=3))


Train Accuracy:  0.7097997538643426
Test Accuracy:  0.3898809523809524

             precision    recall  f1-score   support

          0       0.33      0.38      0.36        13
          1       0.44      0.47      0.45        15
          2       0.38      0.32      0.34        19

avg / total       0.38      0.38      0.38        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### Random Forests

In [15]:
model = RandomForestClassifier(n_estimators=1200, max_depth=10, bootstrap=True, n_jobs=-1)

test_model(model)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  1.0
Test Accuracy:  0.7570283882783883

             precision    recall  f1-score   support

          0       0.50      0.23      0.32        13
          1       0.52      0.73      0.61        15
          2       0.75      0.79      0.77        19

avg / total       0.61      0.62      0.59        47



### Support Vector Machines

In [16]:
test_model(SVC(C=1.0, kernel="rbf", gamma="auto"))


Train Accuracy:  1.0
Test Accuracy:  0.4087087912087912

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        13
          1       0.32      1.00      0.48        15
          2       0.00      0.00      0.00        19

avg / total       0.10      0.32      0.15        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Extremely Randomized Trees

In [17]:
test_model(ExtraTreesClassifier(n_estimators=1200, max_depth=10, bootstrap=True, n_jobs=-1))

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  1.0
Test Accuracy:  0.6875183150183151

             precision    recall  f1-score   support

          0       0.67      0.31      0.42        13
          1       0.65      0.87      0.74        15
          2       0.76      0.84      0.80        19

avg / total       0.70      0.70      0.68        47



### Gradient Boosting Machines

In [18]:
model = GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)

test_model(model)


Train Accuracy:  1.0
Test Accuracy:  0.7340979853479853

             precision    recall  f1-score   support

          0       0.45      0.38      0.42        13
          1       0.57      0.80      0.67        15
          2       0.87      0.68      0.76        19

avg / total       0.66      0.64      0.64        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### XGBoost (Best for now with 78.53%)

In [19]:
# best_model = XGBClassifier(n_estimators=4000, max_depth=20, learning_rate=0.03)

model = XGBClassifier(n_estimators=4000, max_depth=20, learning_rate=0.03, n_jobs=-1)

test_model(model, cv=10)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/


Train Accuracy:  1.0
Test Accuracy:  0.7853296703296703

             precision    recall  f1-score   support

          0       0.67      0.46      0.55        13
          1       0.63      0.80      0.71        15
          2       0.84      0.84      0.84        19

avg / total       0.73      0.72      0.72        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### AdaBoost with Decision Tree

In [20]:
tree = DecisionTreeClassifier()
ada = AdaBoostClassifier(tree, n_estimators=10**7, learning_rate=0.03)

test_model(ada)


Train Accuracy:  1.0
Test Accuracy:  0.6758562271062271

             precision    recall  f1-score   support

          0       0.38      0.46      0.41        13
          1       0.47      0.47      0.47        15
          2       0.62      0.53      0.57        19

avg / total       0.51      0.49      0.49        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### Neural Network

In [21]:
test_model(
    MLPClassifier(
        hidden_layer_sizes=(20, 40, 60, 100, 200, 300, 500, 500, 300, 200, 100, 60, 40, 20),
        activation="logistic"
    )
)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  0.4084551476850277
Test Accuracy:  0.4087087912087912

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        13
          1       0.32      1.00      0.48        15
          2       0.00      0.00      0.00        19

avg / total       0.10      0.32      0.15        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Build up a Random Forest Classifier with Grid Search

In [ ]:
model = RandomForestClassifier()

grid_search = GridSearchCV(
    model,
    param_grid={
        "n_estimators": [100, 200, 300, 500, 700, 1000],
        "max_depth": [1, 2, 3, 5, 10],
    },
    scoring="accuracy",
    cv=3,
    verbose=True
)

grid_search.fit(X, y)

In [71]:
grid_search.best_score_

0.6690140845070423

In [72]:
grid_search.best_params_

{'max_depth': 10, 'n_estimators': 1000}